# Purpose
Update County Tracking table to account for new county entities.

# Setup
## Import - Packages

In [21]:
# ---------------------------------------------------------------------------- #
# GENERAL #
import datetime as dt
import geopandas as gpd
import os
import pathlib as pl
import shutil
import sys

import geopandas as gpd

# ---------------------------------------------------------------------------- #
# ANALYSIS
import numpy as np
import pandas as pd
import siuba as s

# ---------------------------------------------------------------------------- #
# LOCAL #
from tracking_counties.config import SEED, ROOT_DIR

## Import - Tables

In [22]:
tables_path = pl.PurePath(
    ROOT_DIR, 'data', 'tables',
    )
county_path = pl.PurePath(
    tables_path, 'list_of_counties_active.csv'
    )

dct_dtypes = {
    'state_code': str,
    'state_name': str,
    'county_code': str,
    'county_name': str,
    'visited': int,
    # 'date': str,
}

df_visited = pd.read_csv(
    county_path, 
    dtype       = dct_dtypes,
    parse_dates = ['date']
    )

## Import - Shapefiles

In [23]:
sf_path = pl.PurePath(
    ROOT_DIR, 'data', 'shapefiles', '2023_county', 'tl_2023_us_county.shp'
)
df_sf = gpd.read_file(str(sf_path))

# Set column names to lower case
df_sf.columns = [col.lower() for col in df_sf.columns]

## Import - State And Territory Codes

In [24]:
file_path = pl.PurePath(
    tables_path, 'state_and_territory_codes.txt'
)
df_codes = pd.read_csv(file_path, sep='|')

# Set c0lumn names to lower case
df_codes.columns = [col.lower() for col in df_codes.columns]

# Processing

## Create GEOID For Tracking File

In [25]:
def create_geoid(row, state_col = 'state_code', county_col = 'county_code'):
    return f'{row[state_col].zfill(2)}{row[county_col].zfill(3)}'

df_visited['geoid'] = df_visited.apply(
    create_geoid, axis = 1
)

## Join Tracking to Shapefile

In [26]:
df = (
    df_sf
    >> s.left_join(
        s._,
        df_visited,
        by = 'geoid'
    )
)

## Update State and Territory Code Table

In [27]:
# Rename Columns
df_codes = (
    df_codes 
    >> s.rename(
        state_code = s._.state,
        state = s._.stusab
    )
    >> s.select(-s._.statens)
)

# Add leading Zero
df_codes['state_code'] = df_codes['state_code'].apply(
    lambda x: str(x).zfill(2)
)

# Create New Active Record
We only need a new table if we detect new entities have been created

## Check for NAs

In [28]:
n_na = df['state_name'].isna().sum()
print(f'New Records: {n_na:,}')

New Records: 102


## Fill NAs

In [30]:
# Update Visited
df = (
    df
    # >> s.filter(s._.visited.isna())
    >> s.mutate(
        visited = s.if_else(
            s._.visited.isna(),
            0,
            s._.visited
        )
    )
)

# Convert visited to integer
df['visited'] = df['visited'].astype(int)

# Update state code
df['state_code'] = df['geoid'].apply(
    lambda x: x[:2]
)

# Update county code
df['county_code'] = df['geoid'].apply(
    lambda x: x[2:]
)

# Update county name
df['county_name'] = df['name']

# Update state name
df = (
    df
    >> s.select(
        -s._.state,
        -s._.state_name
    )
    >> s.left_join(
        s._,
        df_codes,
        by = 'state_code'
    )
)

# Export

## Log Old Record

In [50]:
str(county_path)

'/Users/evancanfield/Projects/tracking_counties/data/tables/list_of_counties_active.csv'

In [52]:
if n_na > 0:
    # ARCHIVE CURRENT RECORD ---------------------------------------------------
    # Create today's timestamp
    ts = dt.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')

    # Define file path for archive
    new_file_name = f'list_of_counties_active_{ts}.csv'
    archive_path = pl.PurePath(tables_path, 'archive', new_file_name)

    # Move file
    shutil.move(county_path, archive_path)

    # SAVE NEW RECORD ----------------------------------------------------------
    df_output = df[df_visited.columns]
    df_output.to_csv(str(county_path), index = False)
